## Вывод полуаналитических зависимостей для метрики $d_{\text{max}}$ (изохронное отклонение)

Вид линий уровня для графика $r_{min}(\sigma_{pos}, \sigma_{vel})$ поддатлкивает на мысль о том, что зависимость должна быть такой:
$$
r_{min} = \sqrt[n]{\alpha_1\cdot\sigma_{pos}^n+\alpha_2\cdot\sigma_{vel}^n}
$$

Для нахождения коэффициентов $\alpha = (\alpha_1, \alpha_2)$ воспользуемся формулой функционала для линейной регресии:
$$
J_2 = \dfrac{1}{N}\sum_{i=1}^n(r_{min}^n-\alpha_1\cdot\sigma_{pos}^n-\alpha_2\cdot\sigma_{vel}^n)^2 \longrightarrow \min_{\alpha_1, \alpha_2}
$$

Для такого функционала есть ответ. Если переписать его ядро в матричном виде, то по формуле Маркова-Гаусса:
$$
\boxed{\alpha^*=(A^TA)^{-1}A^Ty} = \alpha^*(n)
$$

In [3]:
from utils.libration_sense import get_xf
from utils.formula_creators import n_finder, alpha_finder_of_n

In [2]:
from IPython.display import display, Math

xfinal = get_xf("L2", 290)

n, A_normed, y_du = n_finder("L2", 290, xfinal, grid_density=10)
print('Степень n = ', n, sep='')
alpha, deviation_max = alpha_finder_of_n(A_normed, y_du, n)

IS:  daceypy.array
T:  <class 'numpy.float64'>
Степень n = 1.7867894670541278
